In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.attraction_constants import *
from packages.attraction.Attraction import *

import os
import glob
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions


In [8]:
"""
Formats a search parameter string for Google Maps by replacing spaces with '+'.

Args:
    searchParam: The search parameter string to format.

Returns:
    The formatted search parameter string with spaces replaced by '+'.
"""
def format_searchParam(searchParam) -> str:
    return searchParam.replace(" ", "+")


In [9]:
test_df = pd.read_csv(fh.PATH_TO_GEOCODE)
test_df = test_df[test_df['province_en'] == 'Nakhon Ratchasima']
test_df = test_df[(test_df['district_th'] == 'ปักธงชัย') & (test_df['subDistrict_th'] == 'ตะขบ')]
test_df.reset_index(inplace=True, drop=True)
test_df

,ISO_3166_code,province_th,province_en,zip_code,district_th,district_en,geo_code,subDistrict_th,subDistrict_en
0,30,นครราชสีมา,Nakhon Ratchasima,3014,ปักธงชัย,Pak Thong Chai,301405,ตะขบ,Takhop


In [10]:
print(test_df.loc[0, ['ISO_3166_code', 'zip_code', 'geo_code']])
print(test_df.loc[0, 'zip_code'])

ISO_3166_code        30
zip_code           3014
geo_code         301405
Name: 0, dtype: object
3014


In [11]:
test_df2 = pd.read_csv(fh.PATH_TO_GEOCODE)
test_df2 = test_df2[test_df2['province_en'] == 'Nakhon Ratchasima']
# test_df2 = test_df2.drop(columns=['province_th', 'province_en'], inplace=True)
test_df2.drop(columns=['province_th', 'province_en'], inplace=True)
print(test_df2.district_th.value_counts().index)
# np.unique(test_df2.district_th, axis=0)
np.unique(test_df2.district_th.to_list(), axis=0)

Index(['เมืองนครราชสีมา', 'โนนสูง', 'ปักธงชัย', 'ด่านขุนทด', 'ประทาย',
       'ปากช่อง', 'ครบุรี', 'พิมาย', 'สีคิ้ว', 'สูงเนิน', 'ห้วยแถลง', 'คง',
       'โชคชัย', 'โนนไทย', 'บัวใหญ่', 'ชุมพวง', 'หนองบุญมาก', 'จักราช',
       'ขามสะแกแสง', 'เสิงสาง', 'แก้งสนามนาง', 'ขามทะเลสอ', 'โนนแดง',
       'เฉลิมพระเกียรติ', 'สีดา', 'วังน้ำเขียว', 'พระทองคำ', 'เทพารักษ์',
       'บัวลาย', 'บ้านเหลื่อม', 'เมืองยาง', 'ลำทะเมนชัย'],
      dtype='object', name='district_th')


array(['ขามทะเลสอ', 'ขามสะแกแสง', 'คง', 'ครบุรี', 'จักราช', 'ชุมพวง',
       'ด่านขุนทด', 'บัวลาย', 'บัวใหญ่', 'บ้านเหลื่อม', 'ประทาย',
       'ปักธงชัย', 'ปากช่อง', 'พระทองคำ', 'พิมาย', 'ลำทะเมนชัย',
       'วังน้ำเขียว', 'สีคิ้ว', 'สีดา', 'สูงเนิน', 'หนองบุญมาก',
       'ห้วยแถลง', 'เฉลิมพระเกียรติ', 'เทพารักษ์', 'เมืองนครราชสีมา',
       'เมืองยาง', 'เสิงสาง', 'แก้งสนามนาง', 'โชคชัย', 'โนนสูง', 'โนนแดง',
       'โนนไทย'], dtype='<U15')

In [12]:
geo_code_df = pd.read_csv(fh.PATH_TO_GEOCODE)

cnt_for_test = 0
for cur_province_th, cur_province_en in zip(ALL_PROVINCE_TH, ALL_PROVINCE_ENG):
    # for testing workflow
    if(cnt_for_test == 3):
        break
    cnt_for_test += 1

    # edge_driver_path = "D:\XSectorZ\GitHubRepoes\CEPPTravelScraping\CEPPWebScraping\msedgedriver.exe"
    # service = Service(executable_path=edge_driver_path)
    cur_geo_code_by_province_df = geo_code_df[geo_code_df['province_en'] == cur_province_en]
    cur_geo_code_by_province_df.drop(columns=['province_th', 'province_en'], inplace=True)
    cur_all_searchDistrict_th = np.unique(cur_geo_code_by_province_df.district_th.to_list(), axis=0)
    cur_all_searchDistrict_en = np.unique(cur_geo_code_by_province_df.district_en.to_list(), axis=0)
    
    for cur_search_district_th, cur_search_district_en in zip(cur_all_searchDistrict_th, cur_all_searchDistrict_en):
        driver = webdriver.Chrome()

        #url = "https://thai.tourismthailand.org/Search-result/attraction?destination_id=238&sort_by=datetime_updated_desc&page=1&perpage=15&menu=attraction"
        #url = "https://www.google.com/maps/search/%E0%B8%AA%E0%B8%96%E0%B8%B2%E0%B8%99%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%97%E0%B9%88%E0%B8%AD%E0%B8%87%E0%B9%80%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%A7%E0%B8%A3%E0%B8%B2%E0%B8%8A%E0%B8%9A%E0%B8%B8%E0%B8%A3%E0%B8%B5/@13.5524175,98.9526212,9z/data=!3m1!4b1?entry=ttu"
        # url = "https://www.google.com/maps/search/%E0%B8%97%E0%B9%88%E0%B8%AD%E0%B8%87%E0%B9%80%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%A7%E0%B9%80%E0%B8%82%E0%B8%95%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%99%E0%B8%84%E0%B8%A3/@13.7540829,100.4870287,15z/data=!3m1!4b1?authuser=0&entry=ttu"

        # query_url = "https://www.google.com/maps/search/ท่องเที่ยวนครราชสีมา"
        # query_url = "https://www.google.com/maps/search/ท่องเที่ยวเขตพระนคร"
        # สถาน ที่ ท่องเที่ยว near Mueang Nakhon Ratchasima District, Nakhon Ratchasima
        cur_searchParam = format_searchParam(
            searchParam="สถาน ที่ ท่องเที่ยว near {0} District, {1}".format(cur_search_district_en, cur_province_en)
        )
        cur_query_url = "https://www.google.com/maps/search/{0}".format(cur_searchParam)

        driver.get(cur_query_url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'hfpxzc')))
        list_card = driver.find_elements(By.CLASS_NAME, "hfpxzc")
        action = ActionChains(driver)

        # province = "ราชบุรี"
        province = "กรุงเทพมหานคร"

        patternPhone = re.compile(r'\d{3} \d{3} \d{4}')
        patternPhoneService = re.compile(r'\d{3} \d{3} \d{3}')

        subStrDistict = "อำเภอ"
        subStrSubDistict = "ตำบล"

        if province == "กรุงเทพมหานคร":
            subStrDistict = "เขต"
            subStrSubDistict = "แขวง"

        print("check listCard 1")
        print(len(list_card))
        print(list_card)
        print("**************************")

        # cnt_IsFetchAllCard = 0
        isFetchAllCard = False
        while not isFetchAllCard:
            # print(len(list_card))
            prev_card_len = len(list_card)

            scroll_origin = ScrollOrigin.from_element(list_card[-1])
            action.scroll_from_origin(scroll_origin, 0, 1000).perform()
            time.sleep(const.SCROLL_PAUSE_TIME)

            list_card = driver.find_elements(By.CLASS_NAME, "hfpxzc")
            # ensure that there is no others card element(fetched from google every time scrolling) --> break the loop
            # check whether scorlling untill found section 'คุณมาถึงส่วนท้ายของรายการแล้ว' or 'You've reached the end of the list.'
            reach_end_section_span = driver.find_elements(By.CLASS_NAME, "HlvSq")
            if(len(reach_end_section_span)):
                for cur_span in reach_end_section_span:
                    if(cur_span.text == "คุณมาถึงส่วนท้ายของรายการแล้ว" or cur_span.text == "You've reached the end of the list."):
                        isFetchAllCard = True
                        break

            # ensure that there is no others card element(fetched from google every time scrolling) --> break the loop
            # if prev_card_len == len(list_card):
            #     cnt_IsFetchAllCard += 1
            #     # check for 5 time wheter there is no more card fetching
            #     if cnt_IsFetchAllCard > 5:
            #         break
            # else:
            #     cnt_IsFetchAllCard = 0




        print("check listCard 2")
        print(len(list_card))
        print(list_card)
        print("**************************")

        # read data from card component elements
        # for i in range(len(list_card)):
        #     # define attraction object
        #     cur_attraction = Attraction()
            

        #     print("cur Idx -> ", i)
        #     # scroll_origin = ScrollOrigin.from_element(list_card[i])
        #     # action.scroll_from_origin(scroll_origin, 0, 100).perform()
        #     # action.move_to_element(list_card[i]).perform()

        #     # update 20/7/2024: found problem can not click some card element
        #     # will fix later
        #     try:
        #         list_card[i].click()
        #     except Exception as e:
        #         print('can not click card element at Idx -> {0}'.format(i))
        #         print('when scrape for province -> {0}, district -> {1}'.format(province, district))
        #         continue

        #     time.sleep(5)
        #     soup = BeautifulSoup(driver.page_source, 'html.parser')
        #     name = soup.find_all('h1', class_='DUwDvf lfPIob')
        #     description = soup.find_all('div', class_='PYvSYb')
        #     address = soup.find_all('div', class_='Io6YTe')
        #     # loc = soup.find_all('div', class_='rogA2c')

        #     time.sleep(5)
        #     print(name[0].text)
        #     start_index_lat = driver.current_url.find("!3d") + 3
        #     end_index_lat = driver.current_url.find("!4d")
        #     lat = driver.current_url[start_index_lat:end_index_lat]
        #     start_index_long = driver.current_url.find("!4d") + 3
        #     end_index_long = driver.current_url.find("!", driver.current_url.find("!4d") + 1)
        #     long = driver.current_url[start_index_long:end_index_long]
            
        #     # found some wiered place that doesn't evevn have its address
        #     # skip this case for now...
        #     if(not len(address)):
        #         continue

        #     district = 0
        #     subDirstrict = 0

        #     print("LOCATION: " + lat + " " + long)
        #     if(len(description)):
        #         print("Description :",description[0].text)
        #         cur_attraction.set_description(description[0].text)

        #     if(len(address)):
        #         useData = None
        #         for div in address:
        #             if province in div.text and div.text.find(subStrDistict) != -1:
        #                 useData = div.text.replace(",","").replace("เเ","แ")
                
        #         if(useData != None):
        #             print("Full Address :",useData)
        #             start_address_index = useData.find(subStrSubDistict)
        #             subAddress = useData[start_address_index:]
        #             district = subAddress[subAddress.find(subStrDistict)+len(subStrDistict):subAddress.find(province)].replace(" ","")
        #             subdistrict = subAddress[subAddress.find(subStrSubDistict)+len(subStrSubDistict):subAddress.find(subStrDistict)].replace(" ","")

        #             if district == "เมือง":
        #                 district = district+province

        #             # filter row to find 'ISO_3166_code', 'zip_code', 'geo_code'
        #             filtered_rows = geo_code_df[(geo_code_df['province_th'] == province)&(geo_code_df['district_th'] == district)&(geo_code_df['subDistrict_th'] == subdistrict)]
        #             if not filtered_rows.empty:
        #                 print("province :",filtered_rows.iloc[0, 0],province)
        #                 print("District :",filtered_rows.iloc[0, 3],district)
        #                 print("SubDistrict :",filtered_rows.iloc[0, 6],subdistrict)
        #             else:
        #                 print("province :",province)
        #                 print("District :",district)
        #                 print("SubDistrict :",subdistrict)

        #         #openingDay = soup.find_all('td', class_='ylH6lf')
        #         #openingHour = soup.find_all('td', class_='mxowUb')
        #         score_div = soup.find_all('span', class_='ceNzKf')
        #         if len(score_div):
        #             score = score_div[0].get('aria-label').replace(" ","").replace("ดาว","")
        #             print("Rating score :",score)
        #             review_count = soup.find('span', {'aria-label': lambda x: x and 'รีวิว' in x}).text
        #             print("Rating Count:", review_count)

        #         ticketRating = soup.find_all('div', class_='drwWxc')
        #         if(len(ticketRating)):
        #             print("Ticket Price :",ticketRating[0].text)

        #         divContact = soup.find_all('div', class_='Io6YTe fontBodyMedium kR99db')
                
        #         for div in divContact:
        #             if(re.match(patternPhone,div.text) or re.match(patternPhoneService,div.text)):
        #                 print("Contact :",div.text)
        #                 break

        #         openingHourCheck = soup.find_all("span", class_="HlvSq")

        #         if(len(openingHourCheck) and openingHourCheck[0].text == "ดูเวลาทำการเพิ่มเติม"):
        #             infoOpening = driver.find_elements(By.CLASS_NAME, "HlvSq")
        #             for element in infoOpening:
        #                 element.click()
        #             wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'y0skZc')))
        #             soup = BeautifulSoup(driver.page_source, 'html.parser')

        #         openingTime = soup.find_all("tr", class_="y0skZc")

        #         count = 0

        #         for data in openingTime:
        #             dateDiv = data.find("td", class_="ylH6lf")
        #             timeDiv = data.find("td", class_="mxowUb")
        #             print(dateDiv.text,timeDiv.text)
        #             count += 1
        #             if count == 7:
        #                 break
                
        #         # find tag score using Gemini API


                
                
        #     print("---------------------------")
        # print("werf")
        # # while(True):
        # #     pass
        # # print("asdc")

C:\Users\user\AppData\Local\Temp\ipykernel_22080\2883599345.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_geo_code_by_province_df.drop(columns=['province_th', 'province_en'], inplace=True)


check listCard 1
7
[<selenium.webdriver.remote.webelement.WebElement (session="53ab6763db45d33ce191c52a53a02c91", element="f.FBF9D6F71AC579DD760FC0D4ADD435CD.d.5893158A02D4EAA28674D4D293830E02.e.169")>, <selenium.webdriver.remote.webelement.WebElement (session="53ab6763db45d33ce191c52a53a02c91", element="f.FBF9D6F71AC579DD760FC0D4ADD435CD.d.5893158A02D4EAA28674D4D293830E02.e.170")>, <selenium.webdriver.remote.webelement.WebElement (session="53ab6763db45d33ce191c52a53a02c91", element="f.FBF9D6F71AC579DD760FC0D4ADD435CD.d.5893158A02D4EAA28674D4D293830E02.e.171")>, <selenium.webdriver.remote.webelement.WebElement (session="53ab6763db45d33ce191c52a53a02c91", element="f.FBF9D6F71AC579DD760FC0D4ADD435CD.d.5893158A02D4EAA28674D4D293830E02.e.172")>, <selenium.webdriver.remote.webelement.WebElement (session="53ab6763db45d33ce191c52a53a02c91", element="f.FBF9D6F71AC579DD760FC0D4ADD435CD.d.5893158A02D4EAA28674D4D293830E02.e.173")>, <selenium.webdriver.remote.webelement.WebElement (session="53ab67